In [1]:
!pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 29.1 MB/s eta 0:0

In [3]:
from sklearn.preprocessing import OrdinalEncoder

In [2]:
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [4]:
tf.config.run_functions_eagerly(True)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pickle

# Load the tokenizer from the saved file
tokenizer_load_path = '/content/drive/MyDrive/tokenizer.pkl'
with open(tokenizer_load_path, 'rb') as f:
    tokenizer = pickle.load(f)


In [8]:
DATA_PATH = os.path.join("/content", 'drive', 'MyDrive', 'Cyberbully', 'cyberbullying_tweets.csv')
df = pd.read_csv(DATA_PATH)
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [9]:
ord = OrdinalEncoder()
df[['cyberbullying_type']] = ord.fit_transform(df[["cyberbullying_type"]])
LABEL_CAT = ord.categories_

In [10]:

df = df.astype({'cyberbullying_type':'int32', 'tweet_text':'string'})

In [11]:

tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(df['tweet_text'])

In [12]:
@tf.function
def tokenize(tensor):
  return tokenizer.texts_to_sequences(tensor)

In [13]:
df['tokenized'] = tokenize(df['tweet_text'])

In [14]:
def pad_list(lst, max_len):
    padded = lst + [0] * (max_len - len(lst))
    return padded

#max_len = 300
#df['tokenized'] = df['tokenized'].map(lambda x : pad_list(x, max_len))

In [15]:
from tensorflow.keras.models import load_model

# Define the path where the model is saved
saved_model_path = '/content/drive/MyDrive/Saved_models'

# Load the saved model
loaded_model = tf.saved_model.load(saved_model_path)

In [16]:
def preprocess_input(text):
    text_sequence = tokenizer.texts_to_sequences([text])[0]

    # Pad the tokenized sequence to the maximum length
    padded_sequence = pad_list(text_sequence, 128)

    # Return the preprocessed input as a list
    return [padded_sequence]
    return text

# Define a function to classify input text using the loaded model
def classify_text(input_text):
    # Preprocess the input text
    preprocessed_text = preprocess_input(input_text)

    # Use the loaded model to predict the class probabilities
    class_probabilities = loaded_model(preprocessed_text)

    # Return the predicted class probabilities
    return class_probabilities

# Example usage:
input_text = "How are You? @edcrossfineart We are well..."
predicted_probabilities = classify_text(input_text)
predicted_class = np.argmax(predicted_probabilities)
predicted_class_name = ord.inverse_transform([[predicted_class]])[0][0]
print(predicted_probabilities)
print(predicted_class_name)
probs_sorted = sorted(predicted_probabilities[0], reverse=True)
second_highest_prob = probs_sorted[1]
second_highest_class_index = np.where(predicted_probabilities[0] == second_highest_prob)[0][0]
second_highest_class_name = ord.inverse_transform([[second_highest_class_index]])[0][0]

print(f"Second-highest probability: {second_highest_prob}")
print(f"Second-highest class name: {second_highest_class_name}")

tf.Tensor([[0.00257758 0.00328785 0.04459594 0.39123702 0.5568919  0.00140964]], shape=(1, 6), dtype=float32)
other_cyberbullying
Second-highest probability: 0.3912370204925537
Second-highest class name: not_cyberbullying


In [17]:
from tensorflow.keras.models import load_model

# Define the path where the model is saved
saved_model_path = '/content/drive/MyDrive/BiGRU_saved'

# Load the saved model
loaded_model_BiGRU = tf.saved_model.load(saved_model_path)

In [18]:


# Define a function to preprocess the input text
def preprocess_input(text):
    text_sequence = tokenizer.texts_to_sequences([text])[0]

    # Pad the tokenized sequence to the maximum length
    padded_sequence = pad_list(text_sequence, 128)

    # Return the preprocessed input as a list
    return [padded_sequence]
    return text

# Define a function to classify input text using the loaded model
def classify_text(input_text):
    # Preprocess the input text
    preprocessed_text = preprocess_input(input_text)

    # Use the loaded model to predict the class probabilities
    class_probabilities = loaded_model_BiGRU(preprocessed_text)

    # Return the predicted class probabilities
    return class_probabilities

# Example usage:
input_text = "You're nothing but a pathetic loser, go cry to someone who cares."
predicted_probabilities = classify_text(input_text)
predicted_class = np.argmax(predicted_probabilities)
predicted_class_name = ord.inverse_transform([[predicted_class]])[0][0]
print(predicted_probabilities)
print(predicted_class_name)
#[array(['age', 'ethnicity', 'gender', 'not_cyberbullying',
#'other_cyberbullying', 'religion'], dtype=object)]
probs_sorted = sorted(predicted_probabilities[0], reverse=True)
second_highest_prob = probs_sorted[1]
second_highest_class_index = np.where(predicted_probabilities[0] == second_highest_prob)[0][0]
second_highest_class_name = ord.inverse_transform([[second_highest_class_index]])[0][0]

#print(f"Second-highest probability: {second_highest_prob}")
print(f"Second-highest class name: {second_highest_class_name}")

tf.Tensor(
[[3.7057948e-04 1.4163220e-03 1.1610963e-02 6.0199744e-01 3.8429728e-01
  3.0744864e-04]], shape=(1, 6), dtype=float32)
not_cyberbullying
Second-highest class name: other_cyberbullying


In [19]:
from tensorflow.keras.models import load_model

# Define the path where the model is saved
saved_model_path = '/content/drive/MyDrive/BERT_saved'

# Load the saved model
loaded_model_BERT = tf.saved_model.load(saved_model_path)

In [22]:
import ipywidgets as widgets
from IPython.display import display

# Define the function to preprocess the input text
def preprocess_text(text):
    """Preprocesses text data for BERT input."""
    tokens = tokenizer.texts_to_sequences([text])
    return tokens[0]  # Return the tokenized sequence (list of IDs)

def classify(model, text):
    """Classifies text using the loaded BERT model."""
    preprocessed_text = preprocess_text(text)
    # Convert token IDs back to text using the tokenizer's vocabulary
    text_for_prediction = tokenizer.sequences_to_texts([preprocessed_text])
    # Remove extra dimension from the text tensor
    text_for_prediction = tf.constant(text_for_prediction)  # Convert to tensor
    prediction = model(text_for_prediction, training=False)  # Call the model
    return prediction

def on_button_click(b):
    text_to_classify = text_input.value
    prediction = classify(loaded_model_BERT, text_to_classify)
    #print(f"Predicted class probabilities: {prediction}")
    predicted_class = np.argmax(prediction)
    predicted_class_name = ord.inverse_transform([[predicted_class]])[0][0]
    print(predicted_class_name)

# Load the BERT model
from tensorflow.keras.models import load_model

# Define the path where the model is saved
saved_model_path = '/content/drive/MyDrive/BERT_saved'

# Load the saved model
loaded_model_BERT = tf.saved_model.load(saved_model_path)

# Create input widget
text_input = widgets.Text(
    value='',
    placeholder='Enter text to classify',
    description='Text:',
    disabled=False,
    layout=widgets.Layout(width='50%', height='20px')
)

# Create button widget
classify_button = widgets.Button(description="Classify")

# Assign event handler to the button
classify_button.on_click(on_button_click)

# Display widgets
display(text_input)
display(classify_button)


Text(value='', description='Text:', layout=Layout(height='20px', width='50%'), placeholder='Enter text to clas…

Button(description='Classify', style=ButtonStyle())

not_cyberbullying
other_cyberbullying
not_cyberbullying
other_cyberbullying
ethnicity
age
gender
gender
gender
gender
gender
gender
gender
gender
gender
gender
gender
gender
gender
gender
gender


In [ ]:
"Congratulations on your achievement, you worked really hard for it!"
"Thanks for always being there for me, I really appreciate your friendship."
"You did a great job on your presentation, I learned a lot from it."
"You're such a loser, nobody likes you."
"You're ugly and worthless, go crawl back into your hole."
"school bully thinks he is perfect"
"you are the first one i met who doesn't like dinosaurs"
"Hahaha rape jokes and gay jokes and sexist jokes are so  hilarious!!"


In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.75.152.103


In [78]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.75.152.103:8501

npx: installed 22 in 4.784s
your url is: https://angry-shrimps-bet.loca.lt
2024-03-22 09:48:50.647942: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-22 09:48:54.760 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/ops.py", line 3061, in op_def_for_type
    return self._op_def_cache[type]
KeyError: 'CaseFoldUTF8'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 542, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 14, in <module>
    loaded_model_BERT = tf.saved_model.load(saved_bert_model_path)
  File "/usr/local/lib/pytho

# New Section